# Projeto II - SCC0250
## João Villaça - 10724239
## Sergio Piza  -  9361073

#### Instruções:
- wasd: se movimenta pela cena
- mouse: olha ao redor
- espaço: chuta a bola de futebol

### Configurações de janela, código de shader e linkagem:

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
import time
from PIL import Image

glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1080
largura = 1920
window = glfw.create_window(largura, altura, "Projeto 2", None, None)
glfw.make_context_current(window)

vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(position);
            out_normal = normals;
        }
        """
fragment_code = """
        // parametros da iluminacao ambiente e difusa
        uniform vec3 lightPos1; // define coordenadas de posicao da luz #1
        uniform vec3 lightPos2; // define coordenadas de posicao da luz #2
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        uniform float ki; // coeficiente de reflexao difusa
        uniform float ke; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        
        // parametro com a cor da(s) fonte(s) de iluminacao
        vec3 lightColor = vec3(1.0, 1.0, 1.0);

        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        void main(){
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            ////////////////////////
            // Luz #1 interna
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm1 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // direcao da luz
            float diff1 = max(dot(norm1, lightDir1), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse1 = ki * kd * diff1 * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir1 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir1 = reflect(-lightDir1, norm1); // direcao da reflexao
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ki * ks * spec1 * lightColor;    
            
            
            ////////////////////////
            // Luz #2 externa
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm2 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(lightPos2 - out_fragPos); // direcao da luz
            float diff2 = max(dot(norm2, lightDir2), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse2 = ke * kd * diff2 * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir2 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir2 = reflect(-lightDir2, norm2); // direcao da reflexao
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ke * ks * spec2 * lightColor;    
            
            ////////////////////////
            // Combinando as duas fontes
            ////////////////////////
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse1 + diffuse2 + specular1 + specular2),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")
    
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")
    
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# Carregar wavefront (.obj):

In [2]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando vertices
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

# Carregar textura

In [3]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 20
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    #image_data = img.tobytes("raw", "RGB", 0, -1)
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [4]:
vertices_list = []
textures_coord_list = []
normals_list = []

## Matrizes Model, View e Projection

In [5]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

# Função dinâmica para desenhar cada objeto:

In [6]:
light_sources = []
light_sources_limit = 2

def draw_object(obj, 
                angle=90.0, 
                r_x = 0.0, r_y = 1.0, r_z = 0.0,
                t_x=0.0, t_y=0.0, t_z=0.0,
                s_x = 1.0, s_y = 1.0, s_z = 1.0,
                ka = 0.7, kd = 0.9, ks = 0.9, ns = 64.0, light_source=False, light_source_id=0,
                custom_mat_model = []):
    
    global light_sources, light_sources_limit
    
    mat_model = custom_mat_model if len(custom_mat_model) > 0 else model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu 
    
    loc_ns = glGetUniformLocation(program, "ki") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ki) ### envia ns pra gpu
    
    loc_ns = glGetUniformLocation(program, "ke") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ke) ### envia ns pra gpu
    
    if (light_source):
        ##print(obj)
        #print(f"lightPos{light_sources}")
        if (obj not in light_sources and len(light_sources) <= light_sources_limit):
            light_sources.append(obj)
        
        loc_light_pos = glGetUniformLocation(program, f"lightPos{light_sources.index(obj) + 1}") # recuperando localizacao da variavel lightPos na GPU
        glUniform3f(loc_light_pos, obj['pos']['center_x'], obj['pos']['center_y'], obj['pos']['center_z']) ### posicao da fonte de luz
        
    if (light_source_id != 0):
        loc_light_pos = glGetUniformLocation(program, "lightPos1") # recuperando localizacao da variavel lightPos na GPU
        glUniform3f(loc_light_pos, obj['pos']['center_x'], obj['pos']['center_y'], obj['pos']['center_z']) ### posicao da fonte de luz
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, obj['textid'])

    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, obj['initial_vertice'], obj['obj_size']) ## renderizando

In [7]:
# import glfw
# from OpenGL.GL import *
# import OpenGL.GL.shaders
# import numpy as np
# import glm
# import math
# from PIL import Image

# len([])

### Vamos carregar cada modelo e definir funções para desenhá-los

In [8]:
textid = 0
def load2project(pathname, objname, textname):
    global vertices_list, textures_coord_list, textid
    modelo = load_model_from_file(pathname + '/' + objname + '.obj')
    print('Processando modelo '+ objname +'.obj. Vertice inicial:',len(vertices_list))
    
    res = dict()
    res['initial_vertice'] = len(vertices_list)
    
    pos = {
        'max_x': float('-inf'),
        'max_y': float('-inf'),
        'max_z': float('-inf'),
        'min_x': float('inf'),
        'min_y': float('inf'),
        'min_z': float('inf'),
        'center_x': 0.0,
        'center_y': 0.0,
        'center_z': 0.0
    }
    #print(pos)
    for face in modelo['faces']:
        for vertice_id in face[0]:
            v = modelo['vertices'][vertice_id-1]
            if float(v[0]) > float(pos['max_x']): pos['max_x'] = v[0]
            if float(v[0]) < float(pos['min_x']): pos['min_x'] = v[0]
            if float(v[1]) > float(pos['max_y']): pos['max_y'] = v[1]
            if float(v[1]) < float(pos['min_y']): pos['min_y'] = v[1]
            if float(v[2]) > float(pos['max_z']): pos['max_z'] = v[2]
            if float(v[2]) < float(pos['min_z']): pos['min_z'] = v[2]
                
            vertices_list.append( v )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )
        for normal_id in face[2]:
            normals_list.append( modelo['normals'][normal_id-1] )
    
    print('Processando modelo '+ objname +'.obj. Vertice final:',len(vertices_list))
    res['obj_size'] = len(vertices_list) - res['initial_vertice']
    ### inserindo coordenadas de textura do modelo no vetor de texturas
    
    ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
    
    pos['center_x'] = (float(pos['max_x']) + float(pos['min_x'])) / 2.0
    pos['center_y'] = (float(pos['max_y']) + float(pos['min_y'])) / 2.0
    pos['center_z'] = (float(pos['max_z']) + float(pos['min_z'])) / 2.0
    
    res['pos'] = pos
    
    load_texture_from_file(textid, pathname + '/' + textname)
    print("Textura do modelo " + objname + ' carregada com o id:', textid)
    res['textid'] = textid
    textid = textid + 1
    print(pos)
    
    return res

In [9]:
# lista apenas para lembrar o nome das funçoes draw
draw_functions = []

In [10]:
# Terreno
terreno = load2project('planograma', 'grama', 'dirt.jpg')
def draw_terreno():
    draw_object(terreno,ka=0.4,kd=0.8,ks=0.7)
draw_functions.append('draw_terreno')

Processando modelo grama.obj. Vertice inicial: 0
Processando modelo grama.obj. Vertice final: 6
planograma/dirt.jpg RGB
Textura do modelo grama carregada com o id: 0
{'max_x': '225.627914', 'max_y': '-3.512040', 'max_z': '259.397247', 'min_x': '-240.031204', 'min_y': '-3.512040', 'min_z': '-223.904938', 'center_x': -7.201644999999999, 'center_y': -3.51204, 'center_z': 17.746154500000003}


In [11]:
# Pedras da entrada
cobble = load2project('planocobble', 'cobble', 'cobble.jpg')
def draw_cobble():
    draw_object(cobble,ka=0.4,kd=0.8,ks=0.7)
draw_functions.append('draw_cobble')

Processando modelo cobble.obj. Vertice inicial: 6
Processando modelo cobble.obj. Vertice final: 12
planocobble/cobble.jpg RGB
Textura do modelo cobble carregada com o id: 1
{'max_x': '10.398918', 'max_y': '-3.413201', 'max_z': '40.034821', 'min_x': '-5.240333', 'min_y': '-3.413201', 'min_z': '8.307867', 'center_x': 2.5792925, 'center_y': -3.413201, 'center_z': 24.171344}


In [12]:
rua = load2project('planorua', 'road', 'road.jpg')
def draw_rua():
    draw_object(rua,ka=0.2,kd=0.4,ks=0.5)
draw_functions.append('draw_rua')

Processando modelo road.obj. Vertice inicial: 12
Processando modelo road.obj. Vertice final: 18
planorua/road.jpg RGB
Textura do modelo road carregada com o id: 2
{'max_x': '228.508713', 'max_y': '-2.907825', 'max_z': '59.497547', 'min_x': '-241.161179', 'min_y': '-2.907825', 'min_z': '39.673550', 'center_x': -6.326233000000002, 'center_y': -2.907825, 'center_z': 49.5855485}


In [13]:
ceu = load2project('ceu', 'ceu', 'ceu.jpg')
def draw_ceu():
    draw_object(ceu,ka=0.2,kd=0.3,ks=0.4)
draw_functions.append('draw_ceu')

Processando modelo ceu.obj. Vertice inicial: 18
Processando modelo ceu.obj. Vertice final: 2898
ceu/ceu.jpg RGB
Textura do modelo ceu carregada com o id: 3
{'max_x': '210.377884', 'max_y': '210.377945', 'max_z': '210.377823', 'min_x': '-210.377747', 'min_y': '-210.377945', 'min_z': '-210.377686', 'center_x': 6.849999999758438e-05, 'center_y': 0.0, 'center_z': 6.849999999758438e-05}


In [14]:
cama = load2project('cama', 'cama', 'cama.jpg')
def draw_cama():
    draw_object(cama,ka=0.4,kd=0.5,ks=0.6)
draw_functions.append('draw_cama')

Processando modelo cama.obj. Vertice inicial: 2898
Processando modelo cama.obj. Vertice final: 13905
cama/cama.jpg RGB
Textura do modelo cama carregada com o id: 4
{'max_x': '10.240434', 'max_y': '-2.580297', 'max_z': '-1.248173', 'min_x': '7.411808', 'min_y': '-3.341752', 'min_z': '-7.418717', 'center_x': 8.826121, 'center_y': -2.9610244999999997, 'center_z': -4.333445}


In [15]:
# Discoball
print('Carregando Modelos da Bola de Disco')
discoball_cylinder = load2project('discoball', 'cilinder', 'discoball.jpg')
discoball_sphere = load2project('discoball', 'sphere', 'discoball.jpg')

def draw_disco_cylinder():
    draw_object(discoball_cylinder)
    
def draw_disco_ball():
    draw_object(discoball_sphere)

draw_functions.append('draw_disco_cylinder')
draw_functions.append('draw_disco_ball')

Carregando Modelos da Bola de Disco
Processando modelo cilinder.obj. Vertice inicial: 13905
Processando modelo cilinder.obj. Vertice final: 14277
discoball/discoball.jpg RGB
Textura do modelo cilinder carregada com o id: 5
{'max_x': '2.522663', 'max_y': '10.481915', 'max_z': '-5.990288', 'min_x': '2.475950', 'min_y': '7.531761', 'min_z': '-6.037001', 'center_x': 2.4993065000000003, 'center_y': 9.006838, 'center_z': -6.0136445}
Processando modelo sphere.obj. Vertice inicial: 14277
Processando modelo sphere.obj. Vertice final: 17157
discoball/discoball.jpg RGB
Textura do modelo sphere carregada com o id: 6
{'max_x': '3.500000', 'max_y': '7.600000', 'max_z': '-5.000000', 'min_x': '1.500001', 'min_y': '5.600000', 'min_z': '-6.999999', 'center_x': 2.5000005, 'center_y': 6.6, 'center_z': -5.9999994999999995}


In [16]:
# StreetLamp
print('Carregando Modelos do Poste de Rua')
streetlamp_post = load2project('streetlamp', 'streetlamp', 'streetlamp.png')
streetlamp_light = load2project('streetlamp', 'luz', 'luz.png')
# discoball_sphere = load2project('discoball', 'sphere', 'discoball.jpg')

def draw_streetlamp():
    draw_object(streetlamp_post)
    #draw_object(streetlamp_light,angle=0.0,ka=1,kd=1,ks=1, ns=128, light_source=True)
    
# def draw_disco_ball():
#     draw_object(discoball_sphere)

draw_functions.append('draw_streetlamp')
# draw_functions.append('draw_disco_ball')

Carregando Modelos do Poste de Rua
Processando modelo streetlamp.obj. Vertice inicial: 17157
Processando modelo streetlamp.obj. Vertice final: 19623
streetlamp/streetlamp.png RGB
Textura do modelo streetlamp carregada com o id: 7
{'max_x': '17.560301', 'max_y': '21.194319', 'max_z': '63.693226', 'min_x': '14.427908', 'min_y': '-3.616778', 'min_z': '58.055767', 'center_x': 15.994104499999999, 'center_y': 8.7887705, 'center_z': 60.87449650000001}
Processando modelo luz.obj. Vertice inicial: 19623
Processando modelo luz.obj. Vertice final: 22503
streetlamp/luz.png RGB
Textura do modelo luz carregada com o id: 8
{'max_x': '16.631937', 'max_y': '17.268423', 'max_z': '60.491234', 'min_x': '15.368063', 'min_y': '16.004549', 'min_z': '59.227364', 'center_x': 16.0, 'center_y': 16.636485999999998, 'center_z': 59.859299}


In [17]:
# Casa
print('Carregando Modelos da Casa')
casa = load2project('casa', 'casa', 'casa.png')
casa_chao = load2project('casa', 'chao', 'chao.jpg')

def draw_casa():
    draw_object(casa)
    draw_object(casa_chao)
    
draw_functions.append('draw_casa')

Carregando Modelos da Casa
Processando modelo casa.obj. Vertice inicial: 22503
Processando modelo casa.obj. Vertice final: 23937
casa/casa.png RGBA
Textura do modelo casa carregada com o id: 9
{'max_x': '11.877134', 'max_y': '10.703957', 'max_z': '10.126760', 'min_x': '-6.908408', 'min_y': '-3.969795', 'min_z': '-23.702276', 'center_x': 2.484363, 'center_y': 3.3670810000000007, 'center_z': -6.787758}
Processando modelo chao.obj. Vertice inicial: 23937
Processando modelo chao.obj. Vertice final: 23943
casa/chao.jpg RGB
Textura do modelo chao carregada com o id: 10
{'max_x': '10.510990', 'max_y': '-3.346187', 'max_z': '8.664543', 'min_x': '-5.434722', 'min_y': '-3.346187', 'min_z': '-21.874592', 'center_x': 2.538134, 'center_y': -3.346187, 'center_z': -6.6050245}


In [18]:
bola = load2project('ball', 'ball', 'ball.jpg')
def draw_bola():
    draw_object(bola)
    
draw_functions.append('draw_bola')

Processando modelo ball.obj. Vertice inicial: 23943
Processando modelo ball.obj. Vertice final: 179463
ball/ball.jpg RGB
Textura do modelo ball carregada com o id: 11
{'max_x': '34.001717', 'max_y': '-1.506713', 'max_z': '20.998991', 'min_x': '31.998283', 'min_y': '-3.511339', 'min_z': '19.001011', 'center_x': 33.0, 'center_y': -2.509026, 'center_z': 20.000000999999997}


In [19]:
geladeira = load2project('fridge', 'fridge', 'fridge.jpg')
def draw_geladeira():
    draw_object(geladeira)
    
draw_functions.append('draw_geladeira')

Processando modelo fridge.obj. Vertice inicial: 179463
Processando modelo fridge.obj. Vertice final: 181743
fridge/fridge.jpg RGB
Textura do modelo fridge carregada com o id: 12
{'max_x': '-3.331487', 'max_y': '1.456667', 'max_z': '-5.021071', 'min_x': '-5.492515', 'min_y': '-3.216821', 'min_z': '-6.932312', 'center_x': -4.412001, 'center_y': -0.880077, 'center_z': -5.976691499999999}


In [20]:
arco = load2project('arch', 'arch', 'arch.jpg')
def draw_arco():
    draw_object(arco)
    
draw_functions.append('draw_arco')

Processando modelo arch.obj. Vertice inicial: 181743
Processando modelo arch.obj. Vertice final: 186021
arch/arch.jpg RGB
Textura do modelo arch carregada com o id: 13
{'max_x': '49.029488', 'max_y': '17.759346', 'max_z': '2.756157', 'min_x': '18.005220', 'min_y': '-3.766159', 'min_z': '-2.265148', 'center_x': 33.517354, 'center_y': 6.9965935, 'center_z': 0.24550450000000001}


In [21]:
# Árvore
print('Carregando Modelos da Árvore')
arvore_folhas = load2project('tree', 'leaves', 'leaves.jpg')
arvore_tronco = load2project('tree', 'wood', 'wood.jpg')

def draw_arvore():
    draw_object(arvore_folhas)
    draw_object(arvore_tronco)
    
draw_functions.append('draw_arvore')

Carregando Modelos da Árvore
Processando modelo leaves.obj. Vertice inicial: 186021
Processando modelo leaves.obj. Vertice final: 414477
tree/leaves.jpg RGB
Textura do modelo leaves carregada com o id: 14
{'max_x': '-12.046398', 'max_y': '12.610930', 'max_z': '-3.431961', 'min_x': '-25.182995', 'min_y': '1.890132', 'min_z': '-10.271644', 'center_x': -18.6146965, 'center_y': 7.250531, 'center_z': -6.8518025}
Processando modelo wood.obj. Vertice inicial: 414477
Processando modelo wood.obj. Vertice final: 629517
tree/wood.jpg RGB
Textura do modelo wood carregada com o id: 15
{'max_x': '-15.783471', 'max_y': '7.068793', 'max_z': '-5.502903', 'min_x': '-22.055389', 'min_y': '-3.521483', 'min_z': '-8.302691', 'center_x': -18.919430000000002, 'center_y': 1.7736550000000002, 'center_z': -6.902797}


In [22]:
sofa = load2project('sofa', 'sofa', 'sofa.jpg')
def draw_sofa():
    draw_object(sofa)
    
draw_functions.append('draw_sofa')

Processando modelo sofa.obj. Vertice inicial: 629517
Processando modelo sofa.obj. Vertice final: 653505
sofa/sofa.jpg L
Textura do modelo sofa carregada com o id: 16
{'max_x': '5.551959', 'max_y': '-1.229259', 'max_z': '-10.633554', 'min_x': '-0.835000', 'min_y': '-3.370365', 'min_z': '-13.749717', 'center_x': 2.3584795, 'center_y': -2.299812, 'center_z': -12.1916355}


In [23]:
tv = load2project('tv', 'tv', 'tv.png')
def draw_tv():
    draw_object(tv)
    
draw_functions.append('draw_tv')

Processando modelo tv.obj. Vertice inicial: 653505
Processando modelo tv.obj. Vertice final: 666897
tv/tv.png RGBA
Textura do modelo tv carregada com o id: 17
{'max_x': '6.712265', 'max_y': '1.359798', 'max_z': '-20.589508', 'min_x': '-1.355461', 'min_y': '-3.661088', 'min_z': '-21.019714', 'center_x': 2.678402, 'center_y': -1.150645, 'center_z': -20.804611}


In [24]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)

In [25]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

In [26]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [27]:
cameraPos   = glm.vec3(50.0,  10.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

ki = 0.6
ke = 0.7

polygonal_mode = False

ballinfo = dict()
ballinfo['moving'] = False
ballinfo['acceleration'] = 0
ballinfo['speed'] = 0
ballinfo['direction'] = 1
ballinfo['pos'] = 0
ballinfo['rotation'] = 0


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, ballinfo, ki, ke
    
    cameraSpeed = 0.8
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
#     if key == 80 and action==1 and polygonal_mode==True:
#         polygonal_mode=False
#     else:
#         if key == 80 and action==1 and polygonal_mode==False:
#             polygonal_mode=True
            
            
    if key == 80 and (action==1 or action==2) and ki<1.0: #tecla P
        ki += 0.01
        print(ki)
    if key == 85 and (action==1 or action==2) and ki>0.0: #tecla U
        ki -= 0.01
        print(ki)
    if key == 75 and (action==1 or action==2) and ke<1.0: #tecla K
        ke -= 0.01
        print(ke)
    if key == 76 and (action==1 or action==2) and ke>0.0: #tecla L
        ke += 0.01
        print(ke)

                    
    if key == 32 and action == 1 and ballinfo['moving'] == False:
        ballinfo['moving'] = True
        ballinfo['acceleration'] = 1
        ballinfo['speed'] = 0.1
        ballinfo['direction'] *= -1
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

# for f in draw_functions:
#     print('    ' + f + '()')

### Nesse momento, nós exibimos a janela!

In [28]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [29]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   
t_x = 0
rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    
    ### CONFIGURAÇOES DA CÂMERA PARA NAO SAIR DO CENARIO
    # CHECA SE NÃO SAI DA ESFERA DO CÉU
    if((cameraPos[0]/200)**2+(cameraPos[1]/200)**2+(cameraPos[2]/200)**2 > 1):
        i = np.argmax(abs(cameraPos)) # acha o índice com maior valor absoluto da câmera
        if (cameraPos[i]) > 0: 
            cameraPos[i] -= 5.0
        else:
            cameraPos[i] += 5.0
    
    # CHECA SE NÃO PASSA DO CHÃO
    if(cameraPos[1] < 1.0):
        cameraPos[1] = 1.0

    
    
    #draw_object(obj, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z, custom_mat_model):
    
    draw_terreno()
    draw_cobble()
    draw_rua()
    draw_ceu()
    draw_cama()
    draw_disco_cylinder()
    #draw_disco_ball()
    draw_casa()
    #draw_bola()
    draw_geladeira()
    draw_arco()
    draw_arvore()
    draw_sofa()
    draw_tv()
    
    draw_streetlamp()
    
    ### CONFIGURAÇOES DA BOLA DE FUTEBOL
    # CONFIGURAÇOES DA TRANSLAÇÃO
    if (ballinfo['speed'] == 0 or ballinfo['acceleration'] < 0):
        ballinfo['moving'] = False
    
    if ballinfo['moving']:
        ballinfo['pos'] += ballinfo['speed'] * ballinfo['direction'] * ballinfo['acceleration']
        ballinfo['acceleration'] -= 0.001
        
    # CONFIGURAÇÕES DA ROTAÇÃO COM MATRIZ CUSTOMIZADA
    if (ballinfo['moving']):
        ballinfo['rotation'] += ballinfo['speed'] * 18 * ballinfo['acceleration'] * ballinfo['direction']
    
    angle = math.radians(ballinfo['rotation'])
    matrix_transform_soccerball = glm.mat4(1.0) # instanciando uma matriz identidade
    # aplicando translacao
    matrix_transform_soccerball = glm.translate(
        matrix_transform_soccerball, 
        glm.vec3(
            bola['pos']['center_x'] + ballinfo['pos'] - 5.0 , 
            bola['pos']['center_y'] ,
            bola['pos']['center_z'] - 53.0))    
    # aplicando rotacao
    matrix_transform_soccerball = glm.rotate(matrix_transform_soccerball, angle, glm.vec3(0, 0, -1))
    # aplicando translacao
    matrix_transform_soccerball = glm.translate(
        matrix_transform_soccerball, 
        glm.vec3(
            -bola['pos']['center_x'], 
            -bola['pos']['center_y'],
            -bola['pos']['center_z']))    
    # aplicando escala
    matrix_transform_soccerball = glm.scale(matrix_transform_soccerball, glm.vec3(1, 1, 1))
    matrix_transform_soccerball = np.array(matrix_transform_soccerball).T # pegando a transposta da matriz (glm trabalha com ela invertida)

    draw_object(bola, t_x = ballinfo['pos'], custom_mat_model = matrix_transform_soccerball)
    
    
    ### DESENHANDO ESFERA DA DISCOBALL COM MATRIZ DE TRANSFORMAÇAO CUSTOMIZADA
    angle = math.radians(rotacao_inc)
    matrix_transform_discoball = glm.mat4(1.0) # instanciando uma matriz identidade
    # aplicando translacao
    matrix_transform_discoball = glm.translate(
        matrix_transform_discoball, 
        glm.vec3(
            discoball_sphere['pos']['center_x'] - 8.5, 
            discoball_sphere['pos']['center_y'] + 0.5,
            discoball_sphere['pos']['center_z'] + 3.5))    
    # aplicando rotacao
    matrix_transform_discoball = glm.rotate(matrix_transform_discoball, angle, glm.vec3(0, 1, 0))
    # aplicando translacao
    matrix_transform_discoball = glm.translate(
        matrix_transform_discoball, 
        glm.vec3(
            -discoball_sphere['pos']['center_x'], 
            -discoball_sphere['pos']['center_y'],
            -discoball_sphere['pos']['center_z']))    
    # aplicando escala
    matrix_transform_discoball = glm.scale(matrix_transform_discoball, glm.vec3(1, 1, 1))
    matrix_transform_discoball = np.array(matrix_transform_discoball).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    draw_object(discoball_sphere, custom_mat_model = matrix_transform_discoball,ka=3,kd=3,ks=3, ns=400, light_source=True)
    rotacao_inc += 0.1

    

    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)
    
    # atualizando a posicao da camera/observador na GPU para calculo da reflexao especular
    loc_view_pos = glGetUniformLocation(program, "viewPos") # recuperando localizacao da variavel viewPos na GPU
    glUniform3f(loc_view_pos, cameraPos[0], cameraPos[1], cameraPos[2]) ### posicao da camera/observador (x,y,z)
    
    
    glfw.swap_buffers(window)
#     time.sleep(0.1)



glfw.terminate()

0.59
0.58
0.57
0.5599999999999999
0.5499999999999999
0.5399999999999999
0.5299999999999999
0.5199999999999999
0.5099999999999999
0.4999999999999999
0.4899999999999999
0.47999999999999987
0.46999999999999986
0.45999999999999985
0.44999999999999984
0.43999999999999984
0.4299999999999998
0.4199999999999998
0.4099999999999998
0.3999999999999998
0.3899999999999998
0.3799999999999998
0.3699999999999998
0.35999999999999976
0.34999999999999976
0.33999999999999975
0.32999999999999974
0.31999999999999973
0.3099999999999997
0.2999999999999997
0.2899999999999997
0.2799999999999997
0.2699999999999997
0.2599999999999997
0.24999999999999967
0.23999999999999966
0.22999999999999965
0.21999999999999964
0.20999999999999963
0.19999999999999962
0.1899999999999996
0.1799999999999996
0.1699999999999996
0.1599999999999996
0.14999999999999958
0.13999999999999957
0.12999999999999956
0.11999999999999957
0.10999999999999957
0.09999999999999958
0.08999999999999958
0.07999999999999959
0.06999999999999959
0.05999999